In [77]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import yt
import cmasher as cmr
import matplotlib as mpl
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import AxesGrid, ImageGrid
from numpy import linspace
from cr_covariance2 import cr_spatial_covariance

# plot params
plt.rcParams["xtick.top"] = True 
plt.rcParams["ytick.right"] = True
plt.rcParams["xtick.direction"] = 'in' 
plt.rcParams["ytick.direction"] = 'in' 
plt.rcParams["xtick.minor.visible"] = True 
plt.rcParams["ytick.minor.visible"] = True 
plt.rcParams["xtick.major.size"] = 7
plt.rcParams["xtick.minor.size"] = 4.5
plt.rcParams["ytick.major.size"] = 7
plt.rcParams["ytick.minor.size"] = 4.5
plt.rcParams["xtick.major.width"] = 2
plt.rcParams["xtick.minor.width"] = 1.5
plt.rcParams["ytick.major.width"] = 2
plt.rcParams["ytick.minor.width"] = 1.5
plt.rcParams['axes.linewidth'] = 2
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
#plt.rcParams.update({"text.usetex": True})
#plt.style.use('dark_background')

In [78]:
# turn off silly warnings
yt.set_log_level("error")

In [79]:
#path = "/scratch/gpfs/ms0821/sampson_ramses/production_runs/res256/M3MA2D22/"
#trial_name = "productionM3MA2D22"
#savename = trial_name + ".pdf"

path = "/scratch/gpfs/ms0821/sampson_ramses/production_runs/multi256/M3MA05D23/"
trial_name = "production_multi_M3MA05D23"
savename = trial_name + ".pdf"

#path = "/scratch/gpfs/ms0821/sampson_ramses/production_runs/res128/multiGRID/"
#trial_name = "production_test_M3MA1D23"
#savename = trial_name + ".pdf"

In [ ]:
'''
Note that this routine extracts the eigenvalues per snapshot for the simulation. 
It runs over each cr group storing the eigenvalues in a single larger list which 
can then be averaged over if desired. The files are read, and saved for significantly
faster reading after the first run. First run of a new dataset will be very slow.
'''

meta_list = []
eig_i = []
eig_j = []
eig_k = []
time_vec = []
tol=2e-15
num_snapshots = 3
inj_idx = 61
groups = 9
count = 0

for group_idx in range(1,groups+1):
    
    for idx in range(inj_idx, inj_idx+num_snapshots):
        print(f'at file {idx} of group {group_idx}')
        I, eigs, evec, t = cr_spatial_covariance(path=path, num=idx ,trial_name=trial_name, group=group_idx, normalize=True)
        e_i, e_j, e_k = eigs
        eig_i.append(e_i)
        eig_j.append(e_j)
        eig_k.append(e_k)
        time_vec.append(t)
    # add to meta list
    if count == 0:
        times = time_vec
        count += 1
    meta_list.append(eig_i)
    meta_list.append(eig_j)
    meta_list.append(eig_k)

    
    # reset storage lists 
    eig_i = []
    eig_j = []
    eig_k = []
    time_vec = []
times = np.array(times)

at file 61 of group 1
at file 62 of group 1
at file 63 of group 1
at file 61 of group 2
at file 62 of group 2
at file 63 of group 2
at file 61 of group 3
at file 62 of group 3
at file 63 of group 3
at file 61 of group 4
at file 62 of group 4


In [ ]:
'''
Here we perform the group averaging over all of the different CR subpopulations
'''

for group in range(groups):
    if group == 0:
        ave_eig_i = np.array([meta_list[0]])
        ave_eig_j = np.array([meta_list[1]])
        ave_eig_k = np.array([meta_list[2]])
    else:
        ave_eig_i += np.array([meta_list[group*3]])
        ave_eig_j += np.array([meta_list[group*3+1]])
        ave_eig_k += np.array([meta_list[group*3+2]])
        
ave_eig_i = ave_eig_i / groups
ave_eig_j = ave_eig_j / groups 
ave_eig_k = ave_eig_k / groups 
ave_eig_i = ave_eig_i[0]
ave_eig_j = ave_eig_j[0]
ave_eig_k = ave_eig_k[0]

print(times.shape)
print(ave_eig_i.shape)

In [ ]:
'''
Plotting the eigenvalues as a function of time for all i,j,k components
'''

fig = plt.figure(figsize=(11,3), dpi=140)
plt.subplots_adjust(wspace=0.3,hspace=0.4)
nasa = "#ff4f00"

# plot the beast
p1 = plt.subplot(1,3,1)
plt.plot(times, ave_eig_i, color=nasa,label=r"sim with $D_{cr,\perp}=1 \times 10^{23}$")
plt.ylabel(r'$\lambda_{\parallel}$')
plt.xlabel('t (seconds)')

plt.subplot(1,3,2, sharey=p1)
plt.plot(times, ave_eig_j, color=nasa,label=r"sim with $D_{cr,\perp}=1 \times 10^{23}$")
plt.ylabel(r'$\lambda_{\perp,1}$')
plt.xlabel('t (seconds)')

plt.subplot(1,3,3, sharey=p1)
plt.plot(times, ave_eig_k, color=nasa)
plt.ylabel(r'$\lambda_{\perp,2}$')
plt.xlabel('t (seconds)')

plt.savefig(savename, bbox_inches='tight', pad_inches=0.1, dpi=200)

In [ ]:
# naive gradient measurements -- assuming Gaussian diffusion
gradient = (ave_eig_i[-1] - ave_eig_i[0]) / (times[-1] - times[0]) / 2 # factor to go from MSD to diffusion coef
gradient2 = (ave_eig_j[-1] - ave_eig_j[0]) / (times[-1] - times[0]) / 2
gradient3 = (ave_eig_k[-1] - ave_eig_k[0]) / (times[-1] - times[0]) / 2

print(f"measured diff coef: {gradient}")
print(f"measured diff coef 2: {gradient2}")
print(f"measured diff coef 3: {gradient3}")

In [ ]:
'''
Plotting the derrivitive of the eigenvalues (the diffusion coefficient) as a function of time
'''

# make a derrivitive plot
nasa = "#ff4f00"

de1 = np.diff(ave_eig_i) / np.diff(times)
de2 = np.diff(ave_eig_j) / np.diff(times)
de3 = np.diff(ave_eig_k) / np.diff(times)

fig = plt.figure(figsize=(11,3), dpi=140)
plt.subplots_adjust(wspace=0.3,hspace=0.4)

# plot the beast
p1 = plt.subplot(1,3,1)
plt.plot(times[0:-1], de1, color=nasa,label=r"sim with $D_{cr,\perp}=1 \times 10^{23}$")
plt.ylabel(r'$D_{cr, \parallel}$')
plt.xlabel('t (seconds)')

plt.subplot(1,3,2, sharey=p1)
plt.plot(times[0:-1], de2, color=nasa,label=r"sim with $D_{cr,\perp}=1 \times 10^{23}$")
plt.ylabel(r'$D_{cr, \perp}$')
plt.xlabel('t (seconds)')

plt.subplot(1,3,3, sharey=p1)
plt.plot(times[0:-1], de3, color=nasa)
plt.ylabel(r'$D_{cr, \perp}$')
plt.xlabel('t (seconds)')

savename = trial_name + "_diffusion_coef.pdf"
plt.savefig(savename, bbox_inches='tight', pad_inches=0.1, dpi=200)